In [ ]:
import matplotlib.pyplot as plt
import GridWorld
import numpy as np
import pandas as pd

In [ ]:
robot=GridWorld.grid_world()

In [ ]:
def print_values(V, rows,columns):
    for i in range(rows):
        print("---------------------------")
        for j in range(columns):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")
def print_policy(policy,rows,columns):
    for i in range(rows):
        print("---------------------------")
        for j in range(columns):
            a = policy.get((i,j), ' ')
            print("  %s  |" % a, end="")
        print("")

In [ ]:
ACTION2INT = {a: i for i, a in enumerate(robot.ACTION_SPACE)}
INT2ONEHOT = np.eye(len(robot.ACTION_SPACE))

In [ ]:
ACTION2INT

In [ ]:
INT2ONEHOT

In [ ]:
def one_hot(k):
    return INT2ONEHOT[k]

In [ ]:
def merge_state_action(s, a):
    ai = one_hot(ACTION2INT[a])
    return np.concatenate((s, ai))

In [ ]:
s=[0,0]
a='U'
merge_state_action(s, a)

In [ ]:
def predictV(s,a,w):
    sa = merge_state_action(s, a)
    x = rbf_feature.transform([sa])[0]
    return np.dot(x,w)

In [ ]:
def gradientV(s,a):
    sa = merge_state_action(s, a)
    x = rbf_feature.transform([sa])[0]
    return x

In [ ]:
def predictV_all_actions(s,w):
    values=[]
    for a in robot.ACTION_SPACE:
        values.append(predictV(s,a,w))
    return values

In [ ]:
def epsilon_greedy(s, w, eps=0.1):
    values=[]
    p = np.random.random()
    if p < (1 - eps):
        values = predictV_all_actions(s,w)
        return robot.ACTION_SPACE[np.argmax(values)]
    else:
        return np.random.choice(robot.ACTION_SPACE)

In [ ]:
def gather_samples(n_episodes=10000):
    samples = []
    for i in range(n_episodes):
        s = robot.initial_state()
        done=False
        while not done:
            a=np.random.choice(robot.ACTION_SPACE) #random action
            sa = merge_state_action(s, a)
            samples.append(sa)
            s, r, done= robot.step(a)
    return samples

In [ ]:
from sklearn.kernel_approximation import RBFSampler
samples=gather_samples(n_episodes=10000)
rbf_feature = RBFSampler(n_components=100)
rbf_feature.fit(samples)

In [ ]:
samples

In [ ]:
sa=[0,0,0,0,0,1]
rbf_feature.transform([sa])

In [ ]:
GAMMA=0.9
ALPHA = 0.01
#Initialize weights as zero
#RBFSampler has 100 dimensions as default, hence we have 100 features and weights
w = np.zeros(100) 
n_episodes = 20000
for it in range(n_episodes):
    # begin a new episode
    s=robot.initial_state()
    done=False
    while not done:
        a = epsilon_greedy(s, w, eps=0.1)
        s2, r, done=robot.step(a)
        if done:
            target = r
        else:
            values = predictV_all_actions(s2,w)
            target = r + GAMMA * np.max(values)               
        # update the weights
        g = gradientV(s,a)
        err=target - predictV(s,a,w)
        w += ALPHA * err * g
        # update state
        s = s2      

In [ ]:
# obtain V* and pi*
V = {}
greedy_policy = {}
for s in robot.States:
    if s not in robot.actions.keys():
        V[s] = 0
    else:
        values = predictV_all_actions(s,w)
        V[s] = np.max(values)
        greedy_policy[s] = robot.ACTION_SPACE[np.argmax(values)]

In [ ]:
print_policy(greedy_policy,3,4)

In [ ]:
print_values(V,3,4)